<a href="https://colab.research.google.com/github/A1r33s-thewarman/Segmenting-and-Clustering/blob/main/Segmenting%20and%20Clustering%20Neighborhoods%20in%20New%20York%20City.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [20]:
import folium

In [4]:
!wget https://raw.githubusercontent.com/sahil8700/Segmenting-and-Clustering-Neighborhoods-in-New-York-City/master/newyork_data.json

--2021-07-07 07:25:49--  https://raw.githubusercontent.com/sahil8700/Segmenting-and-Clustering-Neighborhoods-in-New-York-City/master/newyork_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115774 (113K) [text/plain]
Saving to: ‘newyork_data.json’

newyork_data.json   100%[===================>] 113.06K  --.-KB/s    in 0.02s   

2021-07-07 07:25:50 (5.01 MB/s) - ‘newyork_data.json’ saved [115774/115774]



In [5]:
with open("newyork_data.json") as jsondata:
  nycdata = json.load(jsondata)

In [6]:
ngh_data = nycdata['features']
ngh_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [7]:
col = ['borough', 'neighbourhood', 'lat', 'long']
data = pd.DataFrame(columns=col)

In [8]:
data

,borough,neighbourhood,lat,long


In [9]:
for i in ngh_data:
  borough = i['properties']['borough']
  ngh = i['properties']['name']
  location = i['geometry']['coordinates']

  data = data.append({'borough':borough,'neighbourhood':ngh,'lat':location[1],'long':location[0]}, ignore_index=True)

In [15]:
data.shape

(306, 4)

In [16]:
data.head()


,borough,neighbourhood,lat,long
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [17]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent='ny_explore')
location = geolocator.geocode(address)
longitude = location.longitude
latitude = location.latitude

print(latitude, longitude)

40.7127281 -74.0060152


In [22]:
nycmap=folium.Map(location=[latitude,longitude],zoom_start=11)

In [29]:
for lat,lon,neigh,bor in zip(data['lat'],data['long'],data['neighbourhood'],data['borough']):
  label = '{},{}'.format(neigh,bor)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker([lat,lon],
   popup = label,
    color = 'red',
     redius = 1, 
     fill = True, 
     fill_color = '#fff',
      fill_opacity = 0.7, 
      parse_html = False).add_to(nycmap)

In [30]:
nycmap

In [31]:
data['borough'].unique()

array(['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island'],
      dtype=object)

In [32]:
manhattan = data[data['borough'] == 'Manhattan'].reset_index(drop=True)
manhattan.head()

,borough,neighbourhood,lat,long
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [33]:
manhattanmap = folium.Map(location=[latitude,longitude], zoom_start=12)

for lat,lon,neigh,bor in zip(manhattan['lat'],manhattan['long'],manhattan['neighbourhood'],manhattan['borough']):
  label = '{},{}'.format(neigh,bor)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker([lat,lon],
   popup = label,
    color = 'red',
     redius = 1, 
     fill = True, 
     fill_color = '#fff',
      fill_opacity = 0.7, 
      parse_html = False).add_to(manhattanmap)

In [34]:
manhattanmap